# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries

import json
import pandas as pd
import re

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#from sklearn.externals import joblib
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rafaelmata357/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# load data from database
def load_data(database_path):
    engine = create_engine('sqlite:///{}'.format(database_path))
    df = pd.read_sql('SELECT * FROM MESSAGES', engine)
    X = df.message 
    y = df.iloc[:,4:]
    category_names  = list(y.columns)
    return X, y, category_names

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
       
    
        clean_tokens.append(clean_tok)
  
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(max_df=0.5,max_features=5000,tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
database_path = 'salida.db'
X, y, category_names = load_data(database_path)

X_train, X_test, y_train, y_test = train_test_split(X, y)

#train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.5, max_features=5000,
                                 tokenizer=<function tokenize at 0x7feba4e8ae50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

print('accuracy {}'.format((y_pred == y_test.values).mean()))

accuracy 0.9497804563794798


In [11]:

for i in range(len(y_test.values[0,:])):
    print('Target: {}'.format(category_names[i]))
    print('--'*30)
    print(classification_report(y_test.values[:,i], y_pred[:,i]))

Target: related
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.69      0.45      0.55      1513
           1       0.85      0.94      0.89      4990
           2       0.50      0.37      0.43        51

    accuracy                           0.82      6554
   macro avg       0.68      0.59      0.62      6554
weighted avg       0.81      0.82      0.81      6554

Target: request
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5391
           1       0.84      0.50      0.62      1163

    accuracy                           0.89      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.89      0.88      6554

Target: offer
------------------------------------------------------------
              precision    recall  f1-score   support

        

/Users/rafaelmata357/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5981
           1       0.83      0.51      0.63       573

    accuracy                           0.95      6554
   macro avg       0.89      0.75      0.80      6554
weighted avg       0.94      0.95      0.94      6554

Target: clothing
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6449
           1       0.70      0.15      0.25       105

    accuracy                           0.99      6554
   macro avg       0.84      0.58      0.62      6554
weighted avg       0.98      0.99      0.98      6554

Target: money
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6397
           1       0.57      0.03      0.05       157

    accuracy                 

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
database_path = 'salida.db'
X, y, category_names = load_data(database_path)

X_train, X_test, y_train, y_test = train_test_split(X, y)

'''parameters = {'vect__max_df': (0.5, 0.75, 1.0),
              'vect__max_features': (None, 5000, 10000),
              'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [50, 100, 200],
              'clf__estimator__min_samples_split': [2, 3, 4],
               'clf__estimator__criterion': ['gini','entropy']}'''

parameters = {'clf__estimator__min_samples_split': [2, 3, 4]
              }

cv = GridSearchCV(pipeline, param_grid=parameters)



In [219]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(max_df=0.5,
                                                        max_features=5000,
                                                        tokenizer=<function tokenize at 0x7f9e7a970940>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))]),
             param_grid={'clf__estimator__min_samples_split': [2, 3, 4]})

In [195]:
cv.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f9e7a970940>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'estimator__verbose': False,
 'estimator__vect': CountVectorizer(tokenizer=<function tokenize at 0x7f9e7a970940>),
 'estimator__tfidf': TfidfTransformer(),
 'estimator__clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'estimator__vect__analyzer': 'word',
 'estimator__vect__binary': False,
 'estimator__vect__decode_error': 'strict',
 'estimator__vect__dtype': numpy.int64,
 'estimator__vect__encoding': 'utf-8',
 'estimator__vect__input': 'content',
 'estimator__vect__lowercase': True,
 'estimator__vect__max_df': 1.0,
 'estimator__vect__max_features': None,
 'estimator__vect__min_df': 1,
 'estimator__vect__ngram_range': (1, 1),
 'estimator__vect__preprocessor': None,
 'estimator__vect__stop_words': None,
 'es

In [199]:
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9e7a970940>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# predict on test data
y_pred = cv.predict(X_test)
print('accuracy {}'.format((y_pred == y_test.values).mean()))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
from sklearn.linear_model import SGDClassifier

database_path = 'salida.db'
X, y, category_names = load_data(database_path)

X_train, X_test, y_train, y_test = train_test_split(X, y)

sgd = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None))),
               ])
y_train = y_train.drop(['child_alone'],axis=1)
sgd.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7feba4e8ae50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=SGDClassifier(alpha=0.001,
                                                               max_iter=5,
                                                               random_state=42,
                                                               tol=None)))])

### 9. Export your model as a pickle file

In [ ]:

joblib.dump(cv, model_out.pkl, compress = 1)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.